In [66]:
import requests
from collections import defaultdict
import json
from dotenv import load_dotenv
import os
load_dotenv()


tokens = {
    'nil': '0x0eb638648207d00b9025684d13b1cb53806debe4',
    'teather': '0xdac17f958d2ee523a2206206994597c13d831ec7',
    'matic': '0x7D1AfA7B718fb893dB30A3aBc0Cfc608AaCfeBB0',
    'usd': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48',
    'shib': '0x95aD61b0a150d79219dCF64E1E6Cc01f0B64C4cE',
    'uni': '0x1f9840a85d5af5bf1d1762f925bdaddc4201f984',
    'ape': '0x4d224452801aced8b2f0aebe155379bb5d594381'  # issue with APE
}
token = 'nil'
response = requests.get(
    f"""https://api.etherscan.io/api?module=contract&action=getsourcecode&address={tokens[token]}&apikey={os.environ.get('API_KEY')}""")
data = response.json()
contract_name = data['result'][0]['ContractName']
source = data['result'][0]['SourceCode']
with open(f"{token}.txt", "w") as text_file:
    text_file.write(source)
source_split = source.split('\n')

print(contract_name)
len(source_split)


NIL


1684

In [62]:
contract_def = 'contract ' + contract_name

# find extended contracts
extendables = []
for i, line in enumerate(source_split):
    # if 'contract ' + contract_name in line[:len(contract_def)+1]:
    #     # print(i+1,line)
    #     is_idx = line.find('is') + 2
    #     line = line.replace('{','')
    #     extendables = line[is_idx:].split(',')
    #     extendables = [extendable.strip() for extendable in extendables]
    line_split = line.split(' ')
    for word_i, x in enumerate(line_split):
        if word_i == 0 and x.strip() == 'contract' and word_i + 1 < len(line_split):
            extendables.append(line_split[word_i + 1].strip())
    # if line.strip()[:8] == 'contract':
    #     extendables

print(extendables)


modifiers = defaultdict(set)
# find modifiers
stack = 0
curr_contract = None
# modifiers = []
for line_n, line in enumerate(source_split):
    stripped = line.strip()
    for extendable in extendables + [contract_name]:
        if 'contract ' + extendable in line:
            assert stack == 0
            curr_contract = extendable

    prev = ''
    for i, char in enumerate(stripped):
        if i == 0 and char == '*':
            break
        if char == '/' and prev == '/':
            break
        if char == '{':
            stack += 1
        if char == '}':
            stack -= 1
        prev = char

    # stack += line.count('{')
    # stack -= line.count('}')
    if stack == 0:
        curr_contract = None

    if 'modifier' == stripped[:8]:
        assert curr_contract != None
        modifiers[curr_contract].add(stripped[9:stripped.find('(')])

# print(modifiers)
stack = -1
lines = []
functions = defaultdict(set)
src = {}
visibilities = ['private', 'internal', 'external', 'public']
for line_n, line in enumerate(source_split):
    # print(line_n,stack)
    # print(line)
    # print(line)
    stripped = line.strip()
    if 'function' == stripped[:8]:

        stack = 0
        func_name = stripped[9:stripped.find('(')]
        idx = -1
        vis_found = ''
        for visibility in visibilities:
            if stripped.find(visibility) > idx:
                vis_found = visibility
                idx = stripped.find(visibility)

        if idx == -1:
            continue

        mods = stripped[idx+len(vis_found):]
        mods = mods[:mods.find('(')].split(' ')
        for mod in mods:
            if mod in ['', 'view', 'returns', 'pure', 'virtual']:
                continue
            functions[mod].add(func_name)

    prev = ''
    for i, char in enumerate(stripped):
        if i == 0 and char == '*':
            break
        if char == '/' and prev == '/':
            break
        if char == '{':
            stack += 1
        if char == '}':
            stack -= 1
        prev = char

    if stack >= 0:
        lines.append(line)
    if stack == 0:
        stack = -1
        src[func_name] = lines
        lines = []
    
        

for key in functions:
    functions[key] = list(functions[key])
    for i,func in enumerate(functions[key]):
        functions[key][i] = {
            'name':func,
            'source_code':src[func]
        }



# print("This contract extends the following contracts",extendables)
# print('----------------------')
# print("Modifiers per contract")
# for key in modifiers:
#     print("---Contract:",key,'---')
#     for mod in modifiers[key]:
#         print(mod)


# # modifiers
# print('----------------------')

# print("The modifiers are used in these functions")
# for key in functions:
#     print("--- Modifier",key,'----')
#     for func in functions[key]:
#             print(func)

data = {}
data['contracts'] = []
for key in modifiers:
    modifiers_json = []
    for mod in modifiers[key]:
        modifiers_json.append({
        'name':mod,
        'functions':list(functions[mod])
        })
    main = contract_name == key
    data['contracts'].append(
        {
            'name':key,
            'modifiers':modifiers_json,
            'main':main
        }
    )

data

['Ownable', 'NIL']


{'contracts': [{'name': 'Ownable',
   'modifiers': [{'name': 'onlyOwner',
     'functions': [{'name': 'airdrop',
       'source_code': ['    function airdrop(address recipient, uint256 amount) external onlyOwner {\r',
        '        removeAllFee();\r',
        '        _transfer(_msgSender(), recipient, amount * 10**8);\r',
        '        restoreAllFee();\r',
        '    }\r']},
      {'name': 'setBurnFeePercent',
       'source_code': ['    function setBurnFeePercent(uint256 fee) public onlyOwner {\r',
        '        require(fee < 20, "Burn fee cannot be more than 20% of tx amount");\r',
        '        _burnFee = fee;\r',
        '    }\r']},
      {'name': 'transferOwnership',
       'source_code': ['    function transferOwnership(address newOwner) public virtual onlyOwner {\r',
        '        require(\r',
        '            newOwner != address(0),\r',
        '            "Ownable: new owner is the zero address"\r',
        '        );\r',
        '        emit Ownershi

In [65]:
code = """modifier onlyOwner() {
        require(_owner == _msgSender(), "Ownable: caller is not the owner");
        _;
    }
"""

import pandas as pd

df = pd.DataFrame([{
    'code':code,
    'function_name':'onlyOwner'
}])

df.to_csv("functions.csv", index=False)